In [16]:
from PIL import Image, ImageFont, ImageDraw
import numpy as np
import cv2

In [23]:
class Rect:
    def __init__(self, x0, y0, x1, y1, x2, y2, x3, y3):
        self.x0 = x0
        self.y0 = y0
        self.x1 = x1
        self.y1 = y1
        self.x2 = x2
        self.y2 = y2
        self.x3 = x3
        self.y3 = y3

    def __repr__(self):
        return "x0: %s, y0: %s, x1: %s, y1: %s, x2: %s, y2: %s, x3: %s, y3: %s" % \
               (self.x0, self.y0, self.x1, self.y1, self.x2, self.y2, self.x3, self.y3)

In [24]:
class CharImageGenerator:
    def __init__(self, font_file, font_size, image_size, font_color=(0, 0, 0), draw_borders=False):
        """
        :param font_file: string - путь до файла шрифта
        :param font_size: int - размер шрифта
        :param font_color: (b, g, r) - цвет шрифта
        :param image_size: int - размер выходного изображения с текстом
        :param draw_borders: bool - рисовать границы буквы или нет
        """
        self._font_file = font_file
        self._font_size = font_size
        self._font_color = font_color
        self._image_size = image_size
        self._draw_borders = draw_borders

    def char_to_image(self, char):
        """
        Создать картинку с буквой
        :param char: string
        :return: (Image, Rect) - картинка и координаты буквы на ней
        """
        font = ImageFont.truetype(self._font_file, self._font_size)
        width, height = font.getsize(char)
        bg_color = (255, 255, 255, 0)

        x = (self._image_size - width) / 2
        y = (self._image_size - height) / 2

        image = Image.new("RGBA", (self._image_size, self._image_size), bg_color)
        draw = ImageDraw.Draw(image)

        draw.text((x, y), char, self._font_color, font=font)

        frame = np.array(image)
        y_idx, x_idx, _ = np.where(np.not_equal(frame, bg_color))
        x0 = np.min(x_idx)
        x1 = np.max(x_idx)
        y0 = np.min(y_idx)
        y1 = np.max(y_idx)

        if self._draw_borders:
            draw.rectangle(((x0, y0), (x1, y1)), fill="red")
            draw.text((x, y), char, self._font_color, font=font)

        # размытие
        image = Image.fromarray(cv2.blur(np.array(image), (5, 5)))

        return image, Rect(x0, y0, x1, y0, x0, y1,  x1, y1)

    def add_background(self, img, bg_img_path):
        """
        :param img: Image - изображение, которое нужно наложить на фон
        :param bg_img_path: string - путь до изображения с фоном
        :return: Image
        """
        result = Image.new("RGBA", (self._image_size, self._image_size))
        bg_img = Image.open(bg_img_path)
        result.paste(bg_img, (0, 0))
        result.paste(img, (0, 0), img)
        return result

In [25]:
class CharImageGeneratorWithHomography(CharImageGenerator):
    @staticmethod
    def apply_homography(image, rect):
        """
        Применить гомографию.
        :param image: Image - исходное изображение
        :param rect: Rect - четырехугольник буквы в изображении
        :return (Image, Rect):
        """
        # трапеция
        src_points = np.array([[rect.x0, rect.y0],
                                   [rect.x1, rect.y1],
                                   [rect.x2, rect.y2],
                                   [rect.x3, rect.y3]])
    
        dst_points = np.array([[rect.x0 + 10, rect.y0],
                               [rect.x1 - 10, rect.y1],
                               [rect.x2 - 20, rect.y2],
                               [rect.x3 + 20, rect.y3]])
        
        image = np.array(image)
        h, status = cv2.findHomography(src_points, dst_points)
        img_dst = cv2.warpPerspective(image, h, (image.shape[0], image.shape[1]))

        rect_dst = Rect(*dst_points.flatten())

        return Image.fromarray(img_dst), rect_dst 

In [22]:
!dir

 ��� � ���ன�⢥ C ����� ���� Windows
 ��਩�� ����� ⮬�: 3AB8-66A9

 ����ন��� ����� C:\Users\soin\stuff\repos\fonts

2018/04/30  01:24 AM    <DIR>          .
2018/04/30  01:24 AM    <DIR>          ..
2018/03/07  02:28 AM             2,475 .gitignore
2018/04/30  01:23 AM    <DIR>          .idea
2018/04/26  10:15 AM            41,753 bg.png
2018/04/30  01:01 AM               508 demo.py
2018/04/30  01:24 AM            41,753 final.png
2018/03/07  01:04 AM    <DIR>          fonts
2018/03/15  01:16 AM             2,600 img.png
2018/03/15  01:16 AM                13 img.txt
2018/04/30  01:23 AM             7,181 notebook.ipynb
2018/04/26  02:12 AM            70,855 pattern.jpg
2018/04/26  09:29 AM            13,263 qq.png
2018/03/07  02:21 AM    <DIR>          result
               9 䠩���        180,401 ����
               5 �����  14,338,641,920 ���� ᢮�����


In [28]:
# Использование:
generator = CharImageGeneratorWithHomography(font_file="fonts/Stay Wildy.ttf",
                                             font_size=200,
                                             image_size=200,
                                             font_color=(11, 14, 12),
                                             draw_borders=True)

img, rect = generator.char_to_image("Q")
img, rect = generator.apply_homography(img, rect)
img = generator.add_background(img, 'pattern.jpg')
img.save('final.png', format='png')
    